In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9989.000000,10000.000000,10000.000000,9890.000000
mean,7.297602,27.298900,1.034400,38.699798
std,1.291685,6.165838,0.192926,2.539957
min,0.612885,12.000000,1.000000,17.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.257702,50.000000,3.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5150
False    4850
Name: is_male, dtype: int64

In [5]:
df['is_male'].value_counts()/len(df)

True     0.515
False    0.485
Name: is_male, dtype: float64

In [6]:
df.isnull().sum()

weight_pounds       11
is_male              0
mother_age           0
plurality            0
gestation_weeks    110
dtype: int64

In [7]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [8]:
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
39,8.375361,True,32,1,41.0
6132,4.437905,False,28,1,30.0
5986,7.936641,False,44,1,38.0
7682,6.926924,False,34,1,38.0
4910,7.874912,True,31,1,40.0


In [9]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [10]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [11]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [13]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [14]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6670 samples, validate on 742 samples
Epoch 1/10
6670/6670 [==============================] - 2s 366us/sample - loss: 1.5605 - mae: 0.9755 - mse: 1.5605 - val_loss: 1.7681 - val_mae: 1.0601 - val_mse: 1.7681
Epoch 2/10
6670/6670 [==============================] - 1s 201us/sample - loss: 1.4393 - mae: 0.9438 - mse: 1.4393 - val_loss: 1.2418 - val_mae: 0.8679 - val_mse: 1.2418
Epoch 3/10
6670/6670 [==============================] - 1s 169us/sample - loss: 1.3885 - mae: 0.9253 - mse: 1.3885 - val_loss: 3.4946 - val_mae: 1.6015 - val_mse: 3.4946
Epoch 4/10
6670/6670 [==============================] - 1s 161us/sample - loss: 1.3517 - mae: 0.9109 - mse: 1.3517 - val_loss: 1.3021 - val_mae: 0.8948 - val_mse: 1.3021
Epoch 5/10
6670/6670 [==============================] - 1s 165us/sample - loss: 1.3188 - mae: 0.8995 - mse: 1.3188 - val_loss: 1.6158 - val_mae: 1.0188 - val_mse: 1.6158
Epoch 6/10
6670/6670 [==============================] - 1s 152us/sample - loss: 1.2899 - mae: 0.8904 - 